### 데이터 수집

버스정류장 위치 데이터, 주차장데이터, 학교와 학생 수 데이터, 생활인구 데이터, 행정동 별 매출액 데이터 수집

버스정류장 위치 데이터 (날짜(20240101), 정류장 이름, X좌표, Y좌표)

각 분기마다 데이터 수집 1월, 4월, 7월, 10월

연도별 버스정류장 위치 데이터의 좌표만 알고있음 -> 
역지오코딩을 활용해 버스정류장의 X좌표, Y좌표 값을 동으로 변환 -> 
법정동으로 변환이 되는 문제 -> 
법정동을 행정동으로 바꾸는 작업 -> 
법정동과 행정동이 1:1 매치가 되지 않는 문제 발생 -> 폐기
ex. 행정동: 이화동          법정동: 이화동, 연건동, 동숭동
    행정동: 개포1동 개포2동 법정동: 개포동


같은 이름이 두 개 관악구 신사동, 강남구 신사동
좌표를 찍는 방법으로 좌표를 확인 (행정구역별 위경도 좌표 사용) ->
이 때 이름이 중복되는 것을 방지하기 위해 구와 동으로 좌표를 두 번 확인 후 병합
각 분기마다 시행한 후 행정동별 버스 정류장의 개수 데이터 병합

In [ ]:
주차장 데이터 (년도, 개수, 면수)

각 년도의 주차장 개수와 면수 = 각 분기의 주차장 개수와 면수 ->
각 분기마다 행정동별 주차장 개수와 면수 데이터 병합

상반기,하반기 학교와 학생 수 데이터 ()

각 엑셀 자료에서 초중고 학교명, 학생수 추출 후 csv파일로 변환 ->
서울시 초중고 학교명과 도로명주소 데이터 수집 ->
도로명주소를 카카오 API를 사용해 행정동코드로 변환하려는 작업 시도 -> 실패
오류: 데이터 없음.

지오코딩(카카오 API)를 사용해 도로명 주소를 위도,경도 (X좌표, Y좌표)로 변환 ->
연도별 상반기 하반기 csv파일과 지오코딩한 데이터 병합 ->
역지오코딩을 사용해 좌표를 찍고, 구와 동으로 병합을 한 이후, 행정동별 학교의 수와 학생 총합을 구함 ->

상반기 : 1, 2분기 동일 하반기: 3, 4분기 동일 처리

In [ ]:
생활인구 데이터 (날짜, 시간대, 행정동코드, 총생활인구 (내국인, 장기체류외국인(중국인, 비중국인), 단기체류외국인(중국인, 비중국인)))

각 월 데이터를 분기별로 정리 ->
시간대와 행정동코드별로 총생활인구의 평균값 추출 ->
행정동코드별 내국인, 장기체류외국인, 단기체류외국인 평균 후 병합 ->
행정동코드와 행정동을 병합 ->
이 때 없는 행정동 확인

### 전처리 작업
상일1동 -> 상일동<br>
상일2동 -> 강일동<br>
항동 -> 오류2동<br>
개포3동 -> 일원2동<br>
오류 1동 삭제<br>

### 머신러닝 여러가지 시도

SGD Regressor, Lasso, Ridge규제, 년분기에 따라 매출액이 상승하는지 확인, 19년도 자료 버리기, train_test_random, 시계열분석모델

SGD Regressor, Lasso,Ridge 규제
모델의 성능이 생각보다 높지 않았음 ->
기존 Linear Regressor나 Randomforest 모델이 성능이 더 좋게 나옴

년분기에 따라 매출액이 상승하는지 확인, 19년도 자료 버리기 (20년 1분기 ~ 21년 2분기 코로나 이슈)

초창기엔 미래 매출을 예측하는 모델이기에 분기별로 train test set을 나누었음 
(ex. 2019년1분기~2022년4분기 train, 2023년1분기~2024년2분기 test)
-> 성능이 좋지 않았음

년분기에 따라 매출액이 상승하는 지 확인을 했으나, 업종별로 매출액이 나왔기 때문에 확연한 차이를 보이지 못함 (평균의 오류)
-> 중분류별로도 확인을 해보았으나, 코로나에 따른 매출액의 큰 차이를 확인하지 못함
년분기가 상관관계가 높게 나오지 않았기 때문에 19년도 자료를 버리는 것도 큰 의미를 보이지 못함
따라서 시계열분석모델을 사용해봤을 때도 의미있는 값이 나오지 않았음 (시계열 사용 의미 X)

결국 모든 데이터를 사용하고, train_test_set을 random으로 사용하나 매출의 윈도우를 미루기로 결정
(ex.2024년 1분기 데이터 - 2024년 2분기 매출)
-> random_split의 성능이 더 좋았음

년분기 삭제 시도, 행정동 삭제 시도
-> 모델이 년분기와 행정동의 영향을 받았기에 모두 넣는 값의 모델의 성능이 제일 좋았음

카테고리별 LinearRegressor < 카테고리를 다 넣은 RandomForestRegressor
따라서 최종 모델은 '카테고리를 다 넣은 RandomForestRegressor'

feature 중 모델에 영향을 끼치지 못하는 feature 확인
-> 확인결과 단기체류 외국인의 feature가 영향이 적음
-> feature column 제거 -> 성능 향상 